# Backtesting A Trading Strategy Part 2

## How To Retrieve S&P Constituents Historical Data Using Python

## Introduction
Backtesting is a tool to measure the performance of a trading strategy using historical data. The backtesting process consists of three parts: 1. determining the universe of securities where we will invest in (e.g. equity or fixed income? US or emerging markets?); 2. gathering historical data for the universe of securities; and 3. implementing a trading strategy using the historical data collected.  

In the previous article, I illustrated the first step in the backtesting process of determining the universe of stocks, namely the S&P 500, S&P MidCap 400 and S&P SmallCap 600 indices. In this article, I will discuss the second step of the backtesting process of collecting historical data for each constituent of the universe of stocks. 

## Retrieving S&P Constituents Historical Data
### Step By Step
1. Load the S&P tickers which were gathered from the previous article. 
2. Collect the S&P constituents' 5-year historical data using Python package pandas-datareader from the Investors Exchange (IEX). 

You can find the code below on https://github.com/DinodC/backtesting-trading-strategy.

Import packages

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import pickle
import pandas_datareader.data as web

### S&P Constituents Tickers
In this section, we load the lists pickled from the last article.

Set an id for each index

In [2]:
id = ['sp500', 'sp400', 'sp600']

Create a dictionary to map each id to a tickers file

In [3]:
input_file = {'sp500': 'sp500_barchart.pickle',
              'sp400': 'sp400_barchart.pickle', 
              'sp600': 'sp600_barchart.pickle'} 

Define a dictionary to map each id to a tickers list

In [4]:
sp500_tickers = []
sp400_tickers = []
sp600_tickers = []
sp_tickers = {'sp500': sp500_tickers,
              'sp400': sp400_tickers,
              'sp600': sp600_tickers}

Fill the tickers lists

In [5]:
for i in input_file:
    with open(input_file[i], 'rb') as f:
        
        # Update tickers list        
        sp_tickers[i] = pickle.load(f)

        # Sort tickers list
        sp_tickers[i].sort()
        
    f.close()

### S&P Constituents Historical Data 

Define dictionary of historical data

In [6]:
sp500_data = pd.DataFrame()
sp400_data = pd.DataFrame()
sp600_data = pd.DataFrame()
sp_data = {'sp500': sp500_data,
           'sp400': sp400_data,
           'sp600': sp600_data}

Set the start and date of the historical data

In [7]:
start_date = '2015-01-01'
end_date = '2020-01-01'

Set the source Investors Exchange(IEX) to be used

In [8]:
source = 'iex'

Create a dictionary to map each id to an output file

In [9]:
output_file = {'sp500': 'sp500_data.pickle',
               'sp400': 'sp400_data.pickle',
               'sp600': 'sp600_data.pickle'}

Retrieve historical data for each constituent of each S&P index

In [10]:
for i in output_file:
    
    # Retrieve historical data 
    # Note that we set number of tickers to < 100 because DataReader gives error when number of tickers > 100
    data1 = web.DataReader(sp_tickers[i][:98], source, start_date, end_date)
    data2 = web.DataReader(sp_tickers[i][98:198], source, start_date, end_date)
    data3 = web.DataReader(sp_tickers[i][198:298], source, start_date, end_date)
    data4 = web.DataReader(sp_tickers[i][298:398], source, start_date, end_date)
    data5 = web.DataReader(sp_tickers[i][398:498], source, start_date, end_date)
    if i == 'sp400':
        # Concatenate historical data
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5], axis=1, sort=True)
    if i == 'sp500':
        data6 = web.DataReader(sp_tickers[i][498:], source, start_date, end_date)
        # Concatenate historical data        
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5, data6], axis=1, sort=True)
    elif i == 'sp600':
        data6 = web.DataReader(sp_tickers[i][498:598], source, start_date, end_date)
        data7 = web.DataReader(sp_tickers[i][598:], source, start_date, end_date)
        # Concatenate historical data
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5, data6, data7], axis=1, sort=True)        
    else:
        pass
            
    # Convert index to datetime
    sp_data[i].index = pd.to_datetime(sp_data[i].index)
    
    # Save historical data to file
    with open(output_file[i], 'wb') as f:
        pickle.dump(sp_data[i], f)
    f.close()

## Constituents Close Prices

### S&P 500 Index

Look at the dimensions of our DataFrame

In [11]:
sp_data['sp500'].close.shape

(1116, 505)

Check the first rows

In [12]:
sp_data['sp500'].close.head()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,38.9022,51.6157,157.4188,101.1385,55.5960,84.0786,37.31,40.8010,81.3950,72.340,...,31.3286,39.3276,77.6966,50.5866,31.5092,35.8182,47.6149,108.6834,26.6444,41.9300
2015-01-05,38.1733,51.5822,155.3438,98.2893,54.5497,83.3629,37.07,40.8101,80.0207,71.980,...,30.9730,38.6015,75.5707,50.2359,30.8218,33.5890,46.6475,112.7377,25.6460,41.6783
2015-01-06,37.5786,50.7828,155.2346,98.2985,54.2797,83.8183,36.13,40.3466,79.4435,70.530,...,31.1378,38.0468,75.1689,49.6125,30.4093,33.3915,46.0749,111.7820,24.6665,41.2717
2015-01-07,38.0773,50.7540,158.5704,99.6769,56.4735,85.4914,37.28,40.6738,81.1110,71.110,...,31.4066,38.0603,75.9306,50.8984,30.7988,33.6548,47.6017,114.5621,24.8973,42.1236
2015-01-08,39.2187,51.3764,159.9603,103.5067,57.0642,85.6865,38.96,41.5098,82.3478,72.915,...,31.7709,38.9082,77.1944,52.1550,31.6467,33.9088,48.4310,115.7784,25.2504,42.7723


In [13]:
sp_data['sp500'].close[['A', 'AAL', 'AAP', 'AAPL', 'ABBV']].head()

Symbols,A,AAL,AAP,AAPL,ABBV
date,,,,,
2015-01-02,38.9022,51.6157,157.4188,101.1385,55.5960
2015-01-05,38.1733,51.5822,155.3438,98.2893,54.5497
2015-01-06,37.5786,50.7828,155.2346,98.2985,54.2797
2015-01-07,38.0773,50.7540,158.5704,99.6769,56.4735
2015-01-08,39.2187,51.3764,159.9603,103.5067,57.0642


Check the end rows

In [14]:
sp_data['sp500'].close.tail()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2019-06-04,67.95,29.12,154.61,179.64,76.75,82.21,267.06,77.46,177.97,268.71,...,57.78,106.90,73.59,54.40,33.30,77.40,106.97,117.41,44.40,108.12
2019-06-05,68.35,30.36,154.61,182.54,77.06,81.65,268.80,78.69,179.56,272.86,...,59.32,105.60,72.98,55.38,33.42,78.88,107.29,118.54,44.18,108.50
2019-06-06,69.16,30.38,154.90,185.22,77.07,81.75,269.19,80.09,180.40,274.80,...,59.80,106.01,74.31,55.63,34.03,79.15,108.42,120.31,44.24,108.89
2019-06-07,69.52,30.92,155.35,190.15,77.43,83.48,267.87,80.74,182.92,278.16,...,59.43,107.49,74.58,55.94,34.16,79.56,109.07,120.73,43.64,110.06
2019-06-10,70.29,30.76,153.52,192.58,76.95,84.77,272.43,81.27,184.44,280.34,...,59.26,110.88,74.91,57.10,34.69,80.38,108.65,121.71,43.84,110.22


In [15]:
sp_data['sp500'].close[['A', 'AAL', 'AAP', 'AAPL', 'ABBV']].tail()

Symbols,A,AAL,AAP,AAPL,ABBV
date,,,,,
2019-06-04,67.95,29.12,154.61,179.64,76.75
2019-06-05,68.35,30.36,154.61,182.54,77.06
2019-06-06,69.16,30.38,154.90,185.22,77.07
2019-06-07,69.52,30.92,155.35,190.15,77.43
2019-06-10,70.29,30.76,153.52,192.58,76.95


Descriptive stats

In [16]:
sp_data['sp500'].close.describe()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
count,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,...,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000
mean,53.682820,41.423051,146.493998,140.290845,68.852242,85.875797,179.890578,50.444850,124.099296,151.371232,...,40.836168,61.341104,74.788485,53.898061,26.462871,53.403838,68.914116,113.058047,37.937309,62.868532
std,13.849508,6.496939,25.131046,37.315585,17.563078,8.968281,112.738751,12.825554,28.542652,69.639202,...,7.633397,22.403880,4.468653,7.954859,3.160050,16.177875,15.501008,9.558257,10.788513,19.372341
min,32.258600,24.539800,79.168700,85.976800,42.066600,65.718100,36.130000,33.935700,76.988100,69.990000,...,28.223200,34.795000,58.967500,34.178400,18.532600,28.874500,44.181800,89.236100,18.885300,38.438700
25%,40.231325,36.699875,137.239175,107.503100,54.802525,79.292050,94.167500,40.571050,100.310000,91.392500,...,35.321450,43.897275,72.061175,49.581750,23.883800,35.900400,56.202700,106.314000,26.980875,46.284250
50%,51.786950,41.052250,152.678750,135.893350,60.182250,84.772950,125.575000,44.888600,116.497350,126.590000,...,41.031700,56.992350,75.256750,55.800150,26.286500,50.008100,63.047300,113.959300,40.436250,53.209200
75%,66.235725,46.425700,163.304600,170.222875,85.766250,90.887375,276.560000,60.211775,152.661150,224.590000,...,46.205200,70.171850,77.872600,60.087700,28.750200,68.681950,81.002350,120.219250,47.769625,83.072500
max,81.940000,57.586600,199.159900,229.392000,116.445400,107.649700,449.750000,81.270000,184.440000,289.250000,...,59.800000,139.263300,83.828700,67.795300,35.000000,83.549000,109.070000,130.912800,57.139500,110.220000


In [17]:
sp_data['sp500'].close[['A', 'AAL', 'AAP', 'AAPL', 'ABBV']].describe()

Symbols,A,AAL,AAP,AAPL,ABBV
count,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000
mean,53.682820,41.423051,146.493998,140.290845,68.852242
std,13.849508,6.496939,25.131046,37.315585,17.563078
min,32.258600,24.539800,79.168700,85.976800,42.066600
25%,40.231325,36.699875,137.239175,107.503100,54.802525
50%,51.786950,41.052250,152.678750,135.893350,60.182250
75%,66.235725,46.425700,163.304600,170.222875,85.766250
max,81.940000,57.586600,199.159900,229.392000,116.445400


### S&P MidCap 400 Index

Look at the dimensions of our DataFrame

In [18]:
sp_data['sp400'].close.shape

(1128, 400)

Check the first rows

In [19]:
sp_data['sp400'].close.head()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
2015-01-02,30.2075,35.2031,59.98,19.71,30.35,NaN,12.0755,50.6126,43.1325,47.6957,...,24.0506,21.53,46.9623,11.0529,34.2053,25.4410,40.63,454.3187,55.15,77.43
2015-01-05,30.0891,35.4122,59.12,19.26,29.02,NaN,12.2162,49.9490,41.1493,46.9383,...,23.5846,21.26,45.9472,10.8508,33.5884,24.2546,39.37,444.5068,52.53,76.34
2015-01-06,28.9644,35.7803,58.19,18.95,28.73,NaN,12.2601,49.6634,40.9577,46.8953,...,23.6115,19.74,45.6186,10.8784,33.1878,23.5178,37.98,441.8915,52.44,75.79
2015-01-07,29.8424,35.8388,60.63,19.03,29.32,NaN,12.8577,49.9910,40.9290,47.7817,...,23.8266,20.35,46.0439,10.4282,33.8087,23.5752,38.14,443.5531,52.21,77.72
2015-01-08,30.3258,36.0479,61.76,19.01,30.23,NaN,12.2513,50.8982,41.8008,48.7198,...,23.9789,20.32,45.8119,10.3087,34.6899,24.0919,38.35,447.3186,53.83,79.38


In [20]:
sp_data['sp400'].close[['AAN', 'ACC', 'ACHC', 'ACIW', 'ACM']].head()

Symbols,AAN,ACC,ACHC,ACIW,ACM
2015-01-02,30.2075,35.2031,59.98,19.71,30.35
2015-01-05,30.0891,35.4122,59.12,19.26,29.02
2015-01-06,28.9644,35.7803,58.19,18.95,28.73
2015-01-07,29.8424,35.8388,60.63,19.03,29.32
2015-01-08,30.3258,36.0479,61.76,19.01,30.23


Check the end rows

In [21]:
sp_data['sp400'].close.tail()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
2019-06-04,54.88,46.17,33.72,32.10,33.17,18.92,18.52,101.46,69.42,82.76,...,39.82,18.90,111.80,73.10,41.11,13.29,55.28,683.58,31.79,177.08
2019-06-05,54.97,46.97,33.30,32.26,33.29,19.39,18.54,101.80,69.96,84.89,...,40.63,19.04,113.86,73.68,41.08,13.09,53.93,693.19,31.69,181.59
2019-06-06,54.72,47.02,33.20,32.40,33.18,19.75,17.58,101.53,69.16,85.26,...,40.88,18.46,114.32,73.48,41.01,13.21,53.32,692.20,31.57,181.76
2019-06-07,55.72,47.17,33.91,32.34,33.47,20.39,17.50,100.88,69.75,84.99,...,40.84,18.95,113.60,73.28,41.78,13.58,53.78,690.74,31.48,188.37
2019-06-10,59.23,46.81,33.81,32.75,33.45,20.79,16.76,100.83,70.11,84.70,...,40.63,19.40,113.63,71.51,42.55,13.80,55.55,684.74,31.95,194.51


In [22]:
sp_data['sp400'].close[['AAN', 'ACC', 'ACHC', 'ACIW', 'ACM']].tail()

Symbols,AAN,ACC,ACHC,ACIW,ACM
2019-06-04,54.88,46.17,33.72,32.10,33.17
2019-06-05,54.97,46.97,33.30,32.26,33.29
2019-06-06,54.72,47.02,33.20,32.40,33.18
2019-06-07,55.72,47.17,33.91,32.34,33.47
2019-06-10,59.23,46.81,33.81,32.75,33.45


Descriptive stats

In [23]:
sp_data['sp400'].close.describe()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
count,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,655.00000,1116.000000,1116.000000,1116.000000,1116.000000,...,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000
mean,36.350373,39.939185,48.294091,23.224933,32.301595,51.41140,16.391984,80.359913,56.983298,61.714817,...,30.566799,30.454978,65.714888,34.319395,37.721795,23.601209,56.632679,553.035098,36.543020,110.150184
std,9.912823,4.558736,14.509896,3.915154,3.229840,20.70094,3.824099,19.347078,9.236321,12.842531,...,4.380471,24.809183,16.007057,26.167644,7.118456,8.720881,26.018152,63.682948,8.607855,43.313545
min,20.118600,27.793900,24.750000,16.230000,23.150000,12.57000,10.118800,48.962000,40.929000,40.072600,...,22.220300,3.780000,38.906800,9.233700,25.207900,6.455100,19.560000,433.190600,15.230000,46.930000
25%,27.644000,36.077475,37.447500,20.147500,30.187500,31.95000,13.619175,61.302600,48.388325,48.854200,...,27.656025,11.477500,51.121250,16.842825,31.480175,17.791000,35.992500,488.626100,30.380000,77.810000
50%,36.299650,40.569000,44.080000,22.765000,32.415000,57.04690,15.141550,84.654250,57.851800,63.026700,...,31.104450,19.085000,67.425700,20.155750,36.271400,22.852250,49.290000,566.443300,37.235000,100.645000
75%,44.065400,43.776175,59.352500,24.852500,34.440000,66.54120,19.135775,97.804025,65.280100,73.766650,...,33.845575,45.990000,76.265750,39.665300,43.550625,30.006425,69.072500,605.189700,43.542500,142.165000
max,59.230000,48.353300,82.970000,35.520000,40.130000,84.04680,28.421700,113.252900,74.760300,85.260000,...,40.880000,103.090000,114.320000,99.250000,54.884400,45.545800,114.540000,693.190000,57.470000,235.440000


In [24]:
sp_data['sp400'].close[['AAN', 'ACC', 'ACHC', 'ACIW', 'ACM']].describe()

Symbols,AAN,ACC,ACHC,ACIW,ACM
count,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000
mean,36.350373,39.939185,48.294091,23.224933,32.301595
std,9.912823,4.558736,14.509896,3.915154,3.229840
min,20.118600,27.793900,24.750000,16.230000,23.150000
25%,27.644000,36.077475,37.447500,20.147500,30.187500
50%,36.299650,40.569000,44.080000,22.765000,32.415000
75%,44.065400,43.776175,59.352500,24.852500,34.440000
max,59.230000,48.353300,82.970000,35.520000,40.130000


## S&P SmallCap 600 Index

Look at the dimensions of our DataFrame

In [25]:
sp_data['sp600'].close.shape

(1116, 601)

Check the first rows

In [26]:
sp_data['sp600'].close.head()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,10.79,21.1378,36.1497,48.42,26.51,24.3685,76.09,26.1460,NaN,10.20,...,10.0733,23.4646,77.72,10.3510,59.7693,27.5666,NaN,30.8228,17.9012,38.21
2015-01-05,10.65,20.4197,36.5616,46.65,25.69,24.3492,74.00,26.1920,NaN,10.00,...,10.1716,23.6822,78.14,10.4395,58.2492,27.3007,NaN,29.9799,16.2561,38.94
2015-01-06,10.25,20.0968,36.8481,45.66,25.57,23.8478,72.69,26.3208,NaN,9.60,...,10.1195,23.4144,77.59,10.4327,57.1471,26.6455,NaN,28.7633,15.5804,38.46
2015-01-07,9.85,20.2799,37.5466,46.48,25.98,23.6163,75.01,26.7532,NaN,9.72,...,10.4145,23.6738,77.45,10.4667,56.7195,26.9208,NaN,28.6764,14.5325,40.28
2015-01-08,9.96,20.7812,37.7704,48.21,26.63,23.8188,75.50,27.0844,NaN,9.92,...,10.4376,23.6655,78.74,10.3987,57.3086,27.6140,NaN,29.5194,15.6881,41.40


In [27]:
sp_data['sp600'].close[['AAOI', 'AAON', 'AAT', 'AAWW', 'AAXN']].head()

Symbols,AAOI,AAON,AAT,AAWW,AAXN
date,,,,,
2015-01-02,10.79,21.1378,36.1497,48.42,26.51
2015-01-05,10.65,20.4197,36.5616,46.65,25.69
2015-01-06,10.25,20.0968,36.8481,45.66,25.57
2015-01-07,9.85,20.2799,37.5466,46.48,25.98
2015-01-08,9.96,20.7812,37.7704,48.21,26.63


Check the end rows

In [28]:
sp_data['sp600'].close.tail()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
date,,,,,,,,,,,,,,,,,,,,,
2019-06-04,9.19,47.80,45.11,39.70,67.63,37.25,79.67,37.60,35.52,15.30,...,4.08,27.03,137.79,12.54,85.75,28.42,20.95,21.32,13.16,20.65
2019-06-05,9.23,48.08,46.39,38.62,68.00,37.03,77.80,37.28,36.20,14.95,...,4.05,27.46,137.16,12.70,85.79,28.25,21.00,20.70,12.86,19.94
2019-06-06,9.30,48.18,46.39,37.99,67.97,36.90,76.95,39.78,37.06,14.89,...,3.99,27.52,136.06,12.67,86.42,28.32,20.85,20.69,13.01,18.67
2019-06-07,9.50,48.85,46.61,39.88,69.18,36.58,77.95,39.36,38.34,15.03,...,4.05,27.47,136.72,12.68,87.35,28.50,21.01,20.93,12.98,21.65
2019-06-10,9.60,48.64,46.51,40.30,71.91,37.20,78.88,39.47,38.69,15.74,...,4.21,27.29,142.45,12.50,87.97,28.39,21.12,20.57,13.19,21.46


In [29]:
sp_data['sp600'].close[['AAOI', 'AAON', 'AAT', 'AAWW', 'AAXN']].tail()

Symbols,AAOI,AAON,AAT,AAWW,AAXN
date,,,,,
2019-06-04,9.19,47.80,45.11,39.70,67.63
2019-06-05,9.23,48.08,46.39,38.62,68.00
2019-06-06,9.30,48.18,46.39,37.99,67.97
2019-06-07,9.50,48.85,46.61,39.88,69.18
2019-06-10,9.60,48.64,46.51,40.30,71.91


Descriptive stats

In [30]:
sp_data['sp600'].close.describe()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
count,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,152.000000,1116.000000,...,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000,1094.000000,1116.000000,1116.000000,1116.000000
mean,27.743244,30.918709,38.058995,50.943889,33.501694,38.263235,67.166470,33.834044,30.553658,17.025923,...,6.638630,26.699762,75.499521,10.873052,64.134756,26.901846,17.466643,26.470261,18.461373,21.713634
std,17.739201,7.217437,3.162768,9.630046,15.846801,9.777167,10.615101,4.977216,3.460363,6.232940,...,1.439664,2.965398,29.980650,1.427787,11.103416,6.054634,3.218554,6.868666,4.585663,6.338225
min,8.380000,18.515200,30.623200,33.370000,14.500000,21.938400,45.070000,24.986400,20.930200,8.960000,...,3.990000,20.496900,26.700000,7.337200,43.206400,14.741000,10.508700,11.940500,8.155900,11.450000
25%,14.930000,23.764775,36.253125,42.410000,23.020000,29.015975,58.387500,29.762450,28.943125,11.270000,...,5.551425,23.872600,48.457500,9.865475,53.693475,22.452275,14.932475,21.132900,15.543625,17.127500
50%,20.410000,32.401300,37.858000,50.895000,26.100000,37.445250,67.020000,33.295050,30.315400,15.950000,...,6.430150,27.307200,77.015000,10.956150,62.285450,27.334650,17.782900,27.297250,18.884700,20.697500
75%,37.910000,35.719525,39.674850,57.950000,42.792500,46.383700,74.500000,38.071300,32.236275,21.362500,...,7.361450,29.385200,104.135000,11.843275,74.294575,31.442500,19.650875,30.892075,21.879050,24.750000
max,99.610000,51.631400,46.770000,74.000000,74.890000,57.403000,95.540000,43.220900,38.690000,36.625000,...,10.596600,31.602900,142.450000,14.108200,87.970000,39.469500,24.438200,41.762300,30.738900,41.400000


In [31]:
sp_data['sp600'].close[['AAOI', 'AAON', 'AAT', 'AAWW', 'AAXN']].describe()

Symbols,AAOI,AAON,AAT,AAWW,AAXN
count,1116.000000,1116.000000,1116.000000,1116.000000,1116.000000
mean,27.743244,30.918709,38.058995,50.943889,33.501694
std,17.739201,7.217437,3.162768,9.630046,15.846801
min,8.380000,18.515200,30.623200,33.370000,14.500000
25%,14.930000,23.764775,36.253125,42.410000,23.020000
50%,20.410000,32.401300,37.858000,50.895000,26.100000
75%,37.910000,35.719525,39.674850,57.950000,42.792500
max,99.610000,51.631400,46.770000,74.000000,74.890000


## Summary
In this article, we retrieved historical data for every constituent in our universe of stocks - the S&P 500, S&P MidCap 400 and S&P SmallCap 600 indices. The 5-year historical data is relatively straightforward to obtain, and is provided for free by the Investors Exchange. In the next article, we implement a simple trading strategy, and backtest it using the historical data collected.